# Pré-Processamento dos Dados

Ciência e Visualização de Dados em Saúde (2021.1)

UNICAMP - Universidade Estadual de Campinas

Daniel Alvarenga Fernandes, 190943 (Saúde/FCM) \\
Giovane William de Souza Gomes, 093801 (Computação/FEEC) \\
Guilherme Paulino, 117119 (Computação/FEEC) \\
Stephane de Freitas Schwarz, 211518 (Computação/IC)

---




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
root_1 = 'drive/Shareddrives/MO286 - Visualização de Dados/data_csv/raw_sheets/'
!ls 'drive/Shareddrives/MO286 - Visualização de Dados/data_csv/raw_sheets/'

'PLANILHA COM TODOS OS DADOS!.xlsx'   s2.csv   s4.csv   s6.csv	 s8.csv
 s1.csv				      s3.csv   s5.csv   s7.csv	 s9.csv


In [ ]:
root = 'drive/Shareddrives/MO286 - Visualização de Dados/data_csv/'
!ls 'drive/Shareddrives/MO286 - Visualização de Dados/data_csv/'

backup	     mapeamento_nomes.csv  s1.csv  s3.csv  s5.csv  s7.csv  s9.csv
columns.csv  raw_sheets		   s2.csv  s4.csv  s6.csv  s8.csv


In [ ]:
root_imgs = 'drive/Shareddrives/MO286 - Visualização de Dados/data_images/CASOS_CHC_FULL/'

# !ls 'drive/Shareddrives/MO286 - Visualização de Dados/data_images/CASOS_CHC_FULL'

In [ ]:
!pip install unidecode

In [ ]:
# Importação de pacotes
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import unidecode
import glob
import os

In [ ]:
# Removendo colunas vazias
def remove_null_cols(dataset):
  dataset.drop([c for c in dataset if c.startswith('Unnamed')], axis=1, inplace=True)

# Abrir todos os dados
s1 = pd.read_csv(root_1+'s1.csv')
s2 = pd.read_csv(root_1+'s2.csv')
s3 = pd.read_csv(root_1+'s3.csv')
s4 = pd.read_csv(root_1+'s4.csv')
s5 = pd.read_csv(root_1+'s5.csv')
s6 = pd.read_csv(root_1+'s6.csv')
s7 = pd.read_csv(root_1+'s7.csv')
s8 = pd.read_csv(root_1+'s8.csv')
s9 = pd.read_csv(root_1+'s9.csv')

all_datasets = [s1, s2, s3, s4, s5, s6, s7, s8, s9]

for dataset in all_datasets:
  remove_null_cols(dataset)

In [ ]:
# Concatenando todos os dados para visualização de todas as features
dataset = pd.concat([s1, s2], axis=0, ignore_index=True)
dataset = pd.concat([dataset, s3], axis=0, ignore_index=True)
dataset = pd.concat([dataset, s4], axis=0, ignore_index=True)
dataset = pd.concat([dataset, s5], axis=0, ignore_index=True)
dataset = pd.concat([dataset, s6], axis=0, ignore_index=True)
dataset = pd.concat([dataset, s7], axis=0, ignore_index=True)
dataset = pd.concat([dataset, s8], axis=0, ignore_index=True)
dataset = pd.concat([dataset, s9], axis=0, ignore_index=True)

In [ ]:
print('Quantidade total de features: {}'.format(len(list(dataset))))

Quantidade total de features: 60


In [ ]:
# Abrindo mapeamento de colunas
new_col_names = pd.read_csv(root+'columns.csv')
new_col_names.drop_duplicates(subset=['old'], inplace=True)
print('Number of unique columns: ', new_col_names.shape)

Number of unique columns:  (60, 2)


In [ ]:
# Alterando nome das colunas para facilitar acesso
def change_column_names(data, ref):
  dataset = data.copy()
  final_col =[]
  for old_c in dataset.columns:
    new_c = ref.new[ref.old==old_c].values
    if new_c:
      final_col.append(new_c[0])

  return final_col

In [ ]:
def remove_accents(string):
  return unidecode.unidecode(string)

In [ ]:
dataset.columns = change_column_names(dataset, new_col_names)

# Colocando nomes em minúsculo
dataset.paciente = dataset.paciente.str.lower()

print('Quantidade de pacientes: {}'.format(len(np.unique(dataset.paciente.dropna()))))

Quantidade de pacientes: 147


In [ ]:
# Pacientes <-> anonimização geral

dataset = dataset.dropna(subset=['paciente'])
dataset.paciente = dataset.paciente.str.lower()
dataset.paciente = dataset.paciente.str.strip()
dataset.paciente = dataset.paciente.apply(remove_accents)

c = dataset.paciente.astype('category')

d = dict(enumerate(c.cat.categories))

In [ ]:
mapping_names = pd.DataFrame(list(d.items()), columns=['key', 'name'])

In [ ]:
# mapping_names.to_csv(root+'mapeamento_nomes.csv')
mapping_names = pd.read_csv(root+'mapeamento_nomes.csv')

In [ ]:
def name_to_cat(name):
  return int(mapping_names.key[mapping_names.name == name])

In [ ]:
cnt=0
for dataset in all_datasets:
  cnt+=1
  dataset.columns = change_column_names(dataset, new_col_names)

  if 'paciente' not in dataset.columns:

    print("s" + str(cnt) + ".csv: " + "não contém coluna paciente")
    continue

  else:

    try:
      
      dataset.dropna(subset=['paciente'], inplace=True)

      dataset.paciente = dataset.paciente.str.lower()
      dataset.paciente = dataset.paciente.str.strip()
      dataset.paciente = dataset.paciente.apply(remove_accents)
      dataset['paciente_id'] = dataset.paciente.apply(name_to_cat)

    except AttributeError as err:
      print("s" + str(cnt) + ".csv: " + str(err))
      continue



s7.csv: não contém coluna paciente
s9.csv: não contém coluna paciente


## Anonimizar os dados

Serão removidas as colunas de identificação. 

In [ ]:
for dataset in all_datasets:
  dataset.drop(['paciente', 'hc','data_tc','data_tx', 'data_tx_cx','data_nasc'], axis='columns', inplace=True, errors='ignore')

Anonimizando pastas de imagens.

In [ ]:
def get_name_cat(name):

  name = name.lower().strip()
  name = remove_accents(name)
  name = name_to_cat(name)

  return name

In [ ]:
# folders_path = glob.glob(root_imgs+'*')

# for folder_path in folders_path:

#   folder_name = os.path.basename(folder_path)
#   folder = os.path.dirname(folder_path)

#   p_name = get_name_cat(folder_name)

#   print(folder, ' || ', folder_name, ' || ', p_name)
#   break

### Colunas restantes em cada dataset

In [ ]:
s1.head()

,delta_tc_tx_dias,tec_tc,loc_nod,tam_nod_cm,pre_contraste_hiperatenuante,roi_nod_pre_contraste,roi_1_parenquina_adj_pre_contraste,roi_2_parenquina_adj_pre_contraste,aten_med_paranq_pre_cont,hipervascularizacao_arterial,roi_nod_arterial,roi_1_parenquina_adj_arterial,roi_2_parenquina_adj_arterial,aten_med_paranq_arterial,roi_aorta_arterial,washout_portal,roi_nod_portal,roi_1_parenquina_adj_porta,roi_2_parenquina_adj_porta,aten_med_paranq_adj_porta,washout_equi,roi_nod_equi,roi_1_parenquina_adj_equi,roi_2_parenquina_adj_equi,aten_med_paranq_equi,pseudocapsula,gordura_intralesional,invasao_macrovascular,lirads,paciente_id
0,66.0,S,S4,1.9,N,35,47,50,48.5,S,85.0,49.0,53.0,51.0,250.0,N,76.0,74.0,82.0,78.0,S,56.0,61.0,64.0,62.5,N,N,N,5.0,88
1,66.0,S,S7,3.5,N,24,40,48,44,N,49.0,41.0,49.0,45.0,238.0,S,47.0,68.0,72.0,70.0,S,46.0,64.0,59.0,61.5,N,N,N,4.0,88
2,66.0,S,S8,1.7,N,34,47,50,48.5,S,73.0,49.0,53.0,51.0,262.0,S,60.0,75.0,70.0,72.5,S,47.0,58.0,61.0,59.5,N,N,N,5.0,88
3,325.0,S,S7,3.9,N,52,50,45,47.5,S,70.0,53.0,60.0,56.5,347.0,N,95.0,63.0,63.0,63.0,S,63.0,73.0,75.0,74.0,S,N,N,5.0,131
4,325.0,S,S8,2.3,N,46,50,54,52,N,47.0,53.0,60.0,56.5,347.0,S,67.0,63.0,63.0,63.0,S,64.0,73.0,75.0,74.0,S,N,N,4.0,131


In [ ]:
s2.head()

,num_nod,delta_tc_tx_dias,tec_tc,loc_nod,tam_nod_tc_cm,pre_contraste_hiperatenuante,roi_nod_pre_contraste,roi_1_parenquina_adj_pre_contraste,roi_2_parenquina_adj_pre_contraste,aten_med_paranq_pre_cont,hipervascularizacao_arterial,roi_nod_arterial,roi_1_parenquina_adj_arterial,roi_2_parenquina_adj_arterial,aten_med_paranq_arterial,roi_aorta_arterial,washout_portal,roi_nod_portal,roi_1_parenquina_adj_porta,roi_2_parenquina_adj_porta,aten_med_paranq_adj_porta,washout_equi,roi_nod_equi,roi_1_parenquina_adj_equi,roi_2_parenquina_adj_equi,aten_med_paranq_equi,pseudocapsula,gordura_intralesional,invasao_macrovascular,lirads,par_portal,par_equi,ac_arterial,ac_portal,ac_equi,rwr_portal,rwr_equi,aef,tam_nod_ap_cm,invasao_microvascular,grau_histologico,chc_tipo,paciente_id
0,N1,66.0,S,S4,1.9,N,35,47,50,48.5,S,85.0,49.0,53.0,51.0,250.0,N,76.0,74.0,82.0,78.0,S,56.0,61.0,64.0,62.5,N,N,N,5.0,102.6,111.6,34.0,2.0,6.5,10.6,34.1,238.1,3.0,presente,2,não informado,88
1,N2,66.0,S,S7,3.5,N,24,40,48,44,N,49.0,41.0,49.0,45.0,238.0,S,47.0,68.0,72.0,70.0,S,46.0,64.0,59.0,61.5,N,N,N,4.0,148.9,133.7,4.0,23.0,15.5,4.1,6.1,113.6,3.6,presente,3,não informado,88
2,N3,66.0,S,S8,1.7,N,34,47,50,48.5,S,73.0,49.0,53.0,51.0,262.0,S,60.0,75.0,70.0,72.5,S,47.0,58.0,61.0,59.5,N,N,N,5.0,120.8,126.6,22.0,12.5,12.5,17.8,35.6,300.0,0.8,ausente,2,não informado,88
3,N4,170.0,S,S8,2.1,N,18,39,44,41.5,S,33.0,45.0,48.0,46.5,238.0,S,61.0,70.0,77.0,73.5,S,39.0,66.0,68.0,67.0,S,N,N,5.0,120.5,171.8,-13.5,12.5,28.0,-84.8,-18.2,71.4,3.9,presente,não informado,bem diferenciado,6
4,N5,170.0,S,S7,1.6,N,22,39,44,41.5,S,58.0,45.0,48.0,46.5,238.0,S,50.0,70.0,77.0,73.5,S,38.0,66.0,68.0,67.0,S,N,N,5.0,147.0,176.3,11.5,23.5,29.0,13.8,34.5,225.0,não descrito,NaN,NaN,NaN,6


In [ ]:
s3.head()

,num_nod,tam_nod_tc_cm,par_portal,par_equi,ac_portal,ac_equi,rwr_portal,rwr_equi,aef,invasao_microvascular,paciente_id
0,N1,1.9,102.6,111.6,2.0,6.5,10.6,34.1,238.1,presente,88
1,N2,3.5,148.9,133.7,23.0,15.5,4.1,6.1,113.6,presente,88
2,N3,1.7,120.8,126.6,12.5,12.5,17.8,35.6,300.0,ausente,88
3,N72,2.1,120.5,171.8,12.5,28.0,-84.8,-18.2,71.4,presente,6
4,N74,4.1,175.0,131.4,31.5,16.0,27.6,12.1,135.0,ausente,6


In [ ]:
s4.head()

,num_nod,delta_tc_tx_dias,loc_nod,tam_nod_tc_cm,par_portal,par_equi,ac_portal,ac_equi,rwr_portal,rwr_equi,aef,invasao_microvascular,grau_histologico,sexo,idade_tx_cx,causa_hepatopatia,recorrencia,obito,afp_pre,data_afp,paciente_id
0,N1,66,S4,1.9,102.6,111.6,2.0,6.5,10.6,34.1,238.1,presente,2,M,61,VHC,NaN,NaN,NaN,NaN,88
1,N2,66,S7,3.5,148.9,133.7,23.0,15.5,4.1,6.1,113.6,presente,3,M,61,VHC,NaN,NaN,NaN,NaN,88
2,N3,66,S8,1.7,120.8,126.6,12.5,12.5,17.8,35.6,300.0,ausente,2,M,61,VHC,NaN,NaN,NaN,NaN,88
3,N4,170,S8,2.1,120.5,171.8,12.5,28.0,-84.8,-18.2,71.4,presente,não informado,F,36,CRIPTO,NaN,NaN,NaN,NaN,6
4,N6,170,S4/5,4.1,175.0,131.4,31.5,16.0,27.6,12.1,135.0,ausente,não informado,F,36,CRIPTO,NaN,NaN,NaN,NaN,6


In [ ]:
s5.head()

,num_nod,delta_tc_tx_dias,loc_nod,tam_nod_tc_cm,par_portal,par_equi,ac_arterial,ac_portal,ac_equi,rwr_portal,rwr_equi,aef,invasao_microvascular,grau_histologico,sexo,idade_tx_cx,causa_hepatopatia,recorrencia,obito,afp_pre,data_afp,paciente_id
0,N1,66,S4,1.9,102.6,111.6,34.0,2.0,6.5,10.6,34.1,238.1,presente,2,M,61,VHC,NaN,NaN,NaN,NaN,88
1,N2,66,S7,3.5,148.9,133.7,4.0,23.0,15.5,4.1,6.1,113.6,presente,3,M,61,VHC,NaN,NaN,NaN,NaN,88
2,N3,66,S8,1.7,120.8,126.6,22.0,12.5,12.5,17.8,35.6,300.0,ausente,2,M,61,VHC,NaN,NaN,NaN,NaN,88
3,N4,170,S8,2.1,120.5,171.8,-13.5,12.5,28.0,-84.8,-18.2,71.4,presente,não informado,F,36,CRIPTO,NaN,NaN,NaN,NaN,6
4,N6,170,S4/5,4.1,175.0,131.4,11.5,31.5,16.0,27.6,12.1,135.0,ausente,não informado,F,36,CRIPTO,NaN,NaN,NaN,NaN,6


In [ ]:
s6.head()

,num_nod,delta_tc_tx_dias,loc_nod,tam_nod_tc_cm,par_portal,par_equi,ac_portal,ac_equi,rwr_portal,rwr_equi,aef,invasao_microvascular,grau_histologico,sexo,idade_tx_cx,causa_hepatopatia,recorrencia,obito,afp_pre,data_afp,paciente_id
0,N1,66,S4,1.9,102.6,111.6,2.0,6.5,10.6,34.1,238.1,presente,2,M,61,VHC,NaN,NaN,NaN,NaN,88
1,N4,170,S8,2.1,120.5,171.8,12.5,28.0,-84.8,-18.2,71.4,presente,não informado,F,36,CRIPTO,NaN,NaN,NaN,NaN,6
2,N9,325,S7,3.9,66.3,117.5,-32.0,11.0,-35.7,10.0,163.6,ausente,2,M,70,VHC,NaN,NaN,NaN,NaN,131
3,N11,129,S3,1.6,89.9,116.7,-9.0,10.0,-12.7,24.1,165.5,ausente,1,M,61,OH,NaN,NaN,NaN,NaN,116
4,N12,45,S8,4.7,104.8,97.6,4.0,-1.5,-90.9,-43.2,9.5,ausente,3,M,59,VHC,NaN,NaN,NaN,NaN,124


In [ ]:
s7.head()

,tam_nod_tc_cm
0,1.9
1,3.5
2,1.7
3,2.1
4,4.1


In [ ]:
s8.head()

,num_nod,num_lesoes,paciente_id
0,N1,3,88
1,N4,5,6
2,N9,2,131
3,N11,1,116
4,N12,3,124


In [ ]:
s9.head()

,hipervasc,washout_portal,washout_equi,roi_nod_arterial,aten_med_paranq_arterial,roi_nod_portal,aten_med_paranq_adj_porta,roi_nod_equi,aten_med_paranq_equi
0,S,N,S,85,51.0,76,78.0,56,62.5
1,N,S,S,49,45.0,47,70.0,46,61.5
2,S,S,S,73,51.0,60,72.5,47,59.5
3,S,S,S,33,46.5,61,73.5,39,67.0
4,S,S,S,58,46.5,42,73.5,51,67.0


## Analisar dados faltantes

In [ ]:
# Dados faltantes em todas as linhas da coluna
cnt = 0
print('Colunas onde todos os dados estão faltando')
for dataset in all_datasets:
  cnt += 1
  missing_data = dataset.columns[dataset.isna().all()]
  if not missing_data.empty:
    print('s' + str(cnt) + '.csv:', *missing_data, sep='\n- ')

Colunas onde todos os dados estão faltando
s4.csv:
- recorrencia
- obito
- afp_pre
- data_afp
s5.csv:
- recorrencia
- obito
- afp_pre
- data_afp
s6.csv:
- recorrencia
- obito
- afp_pre
- data_afp


In [ ]:
# Remover colunas onde todos os valores são nulos
for dataset in all_datasets:
  dataset.dropna(axis=1, how='all', inplace=True)

## Mapeando valores categóricos

In [ ]:
# Alterando dados de , para .

# dataset.tam_nod_tc_cm = dataset.tam_nod_tc_cm.str.replace(',','.').astype(float)
# dataset.par_portal = dataset.par_portal.str.replace(',','.').astype(float)
# dataset.par_equi = dataset.par_equi.str.replace(',','.').astype(float)
# dataset.ac_arterial = dataset.ac_arterial.str.replace(',','.').astype(float)
# dataset.ac_portal = dataset.ac_portal.str.replace(',','.').astype(float)
# dataset.ac_equi = dataset.ac_equi.str.replace(',','.').astype(float)
# dataset.rwr_portal = dataset.rwr_portal.str.replace(',','.').astype(float)
# dataset.rwr_equi = dataset.rwr_equi.str.replace(',','.').astype(float)
# dataset.aef = dataset.aef.str.replace(',','.').astype(float)

In [ ]:
cnt = 0
for dataset in all_datasets:
  cnt += 1
  try:
    dataset.sexo = dataset.sexo.str.replace('M','0').replace('F', '1').astype(int)
    dataset.invasao_microvascular = dataset.invasao_microvascular.str.replace('ausente','0').replace('presente', '1').astype(int)
  except AttributeError as err:
    print("s" + str(cnt) + ".csv: " + str(err))
    continue

s1.csv: 'DataFrame' object has no attribute 'sexo'
s2.csv: 'DataFrame' object has no attribute 'sexo'
s3.csv: 'DataFrame' object has no attribute 'sexo'
s7.csv: 'DataFrame' object has no attribute 'sexo'
s8.csv: 'DataFrame' object has no attribute 'sexo'
s9.csv: 'DataFrame' object has no attribute 'sexo'


Mapeamento dos valores do grau histológico do paciente

*   1 $\implies 1$
*   2 $\implies 2$
*   3 $\implies 3$
*   4 $\implies 4$
*   1, 2 $\implies 5$
*   1, 3 $\implies 6$
*   1, 4 $\implies 7$
*   2, 3 $\implies 8$
*   2, 4 $\implies 9$
*   3, 4 $\implies 10$
*   sem grau $\implies 11$
*   quimioembolização $\implies 12$
*   não informado $\implies 44$
*   nan $\implies 44$

In [ ]:
cnt = 0
for dataset in all_datasets:
  cnt += 1
  try:
    dataset.grau_histologico = dataset.grau_histologico.replace('sem grau','11').replace('não informado', '44').replace(np.nan, '44')
    dataset.grau_histologico = dataset.grau_histologico.replace('1 e 2', '5').replace('1 e 3', '6').replace('1 e 4', '7')
    dataset.grau_histologico = dataset.grau_histologico.replace('2 e 3', '8').replace('2 e 4', '9').replace('3 e 4', '10')
    dataset.grau_histologico = dataset.grau_histologico.replace('quimioembolização','12').astype(int)
  except AttributeError as err:
    print("s" + str(cnt) + ".csv: " + str(err))
    continue
  except ValueError as err:
    print("s" + str(cnt) + ".csv: " + str(err))
    continue
  

s1.csv: 'DataFrame' object has no attribute 'grau_histologico'
s2.csv: invalid literal for int() with base 10: 'não se aplica'
s3.csv: 'DataFrame' object has no attribute 'grau_histologico'
s7.csv: 'DataFrame' object has no attribute 'grau_histologico'
s8.csv: 'DataFrame' object has no attribute 'grau_histologico'
s9.csv: 'DataFrame' object has no attribute 'grau_histologico'


Mapeamento da causa hepatopatia da lesão

*   CRIPTO $\implies 1$
*   OH $\implies 2$
*   OH + VHB $\implies 3$
*   OH + VHC $\implies 4$
*   VHB $\implies 5$
*   VHC $\implies 6$
*   VHC + OH $\implies 7$
*   VHC + VHB $\implies 8$
*   NaN $\implies 44$

In [ ]:
cnt = 0
for dataset in all_datasets:
  cnt += 1
  try:
    dataset.causa_hepatopatia = dataset.causa_hepatopatia.replace(np.nan, '44')
    dataset.causa_hepatopatia = dataset.causa_hepatopatia.replace('CRIPTO', '1').replace('OH', '2')
    dataset.causa_hepatopatia = dataset.causa_hepatopatia.replace('OH + VHB', '3').replace('OH + VHC', '4')
    dataset.causa_hepatopatia = dataset.causa_hepatopatia.replace('VHB', '5').replace('VHC', '6')
    dataset.causa_hepatopatia = dataset.causa_hepatopatia.replace('VHC + OH', '7').replace('VHC + VHB', '8').astype(int)
  except AttributeError as err:
    print("s" + str(cnt) + ".csv: " + str(err))
    continue

s1.csv: 'DataFrame' object has no attribute 'causa_hepatopatia'
s2.csv: 'DataFrame' object has no attribute 'causa_hepatopatia'
s3.csv: 'DataFrame' object has no attribute 'causa_hepatopatia'
s7.csv: 'DataFrame' object has no attribute 'causa_hepatopatia'
s8.csv: 'DataFrame' object has no attribute 'causa_hepatopatia'
s9.csv: 'DataFrame' object has no attribute 'causa_hepatopatia'


## Salvando os Dados Pré-processados

In [ ]:
s1.to_csv(root+'s1.csv', index=False)
s2.to_csv(root+'s2.csv', index=False)
s3.to_csv(root+'s3.csv', index=False)
s4.to_csv(root+'s4.csv', index=False)
s5.to_csv(root+'s5.csv', index=False)
s6.to_csv(root+'s6.csv', index=False)
s7.to_csv(root+'s7.csv', index=False)
s8.to_csv(root+'s8.csv', index=False)
s9.to_csv(root+'s9.csv', index=False)

## Renoveando diretórios

In [ ]:
import os
import pandas as pd
import numpy as np
from glob import glob
import unidecode

In [ ]:
root_images =  'drive/Shareddrives/MO286 - Visualização de Dados/data_images/CASOS_CHC_FULL/*'	
root_csv =  'drive/Shareddrives/MO286 - Visualização de Dados/data_csv/'
image_csv = 'drive/Shareddrives/MO286 - Visualização de Dados/data_csv/mapeamento_nomes.csv'		

In [ ]:
dataset = pd.read_csv(root_csv+'s5.csv')
mapping_names = pd.read_csv(image_csv)

In [ ]:
def remove_accents(string):
  return unidecode.unidecode(string)

In [ ]:
def name_to_cat(name):
  try:
    return int(mapping_names.key[mapping_names.name == name])
  except:
    return 'exit'

In [ ]:
# Rename folders

name_backup = []

img_root = 'drive/Shareddrives/MO286 - Visualização de Dados/data_images/CASOS_CHC_FULL/'

img_folders = glob(root_images)

for folder_name in img_folders:

  path_ = os.path.dirname(folder_name)
  name_ = os.path.basename(folder_name)
  name = name_.strip().lower()
  name = remove_accents(name)

  name_to_code = name_to_cat(name)

  if name_to_code == 'exit':   
    continue

  name_backup.append(folder_name)

  os.rename(folder_name,  path_+'/'+str(name_to_code))